# 4 - Convolutional Sentiment Analysis

In the previous notebooks, we managed to achieve a test accuracy of ~85% using RNNs and an implementation of the [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759) model. In this final notebook, we will be using a *convolutional neural network* (CNN) to conduct sentiment analysis, implementing the model from [Convolutional Neural Networks for Sentence Classification](https://arxiv.org/abs/1408.5882).

**Note**: I am not aiming to give a comprehensive introduction and explanation of CNNs. For a better and more in-depth explanation check out [here](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/) and [here](https://cs231n.github.io/convolutional-networks/).

Traditionally, CNNs are used to analyse images and are made up of one or more *convolutional* layers, followed by one or more linear layers. The convolutional layers use filters (also called *kernels* or *receptive fields*) which scan across an image and produce a processed version of the image. This processed version of the image can be fed into another convolutional layer or a linear layer. Each filter has a shape, e.g. a 3x3 filter covers a 3 pixel wide and 3 pixel high area of the image, and each element of the filter has a weight associated with it, the 3x3 filter would have 9 weights. In traditional image processing these weights were specified by hand by engineers, however the main advantage of the convolutional layers in neural networks is that these weights are learned via backpropagation. 

The intuitive idea behind learning the weights is that your convolutional layers act like *feature extractors*, extracting parts of the image that are most important for your CNN's goal, e.g. if using a CNN to detect faces in an image, the CNN may be looking for features such as the existance of a nose, mouth or a pair of eyes in the image.

So why use CNNs on text? In the same way that a 3x3 filter can look over a patch of an image, a 1x2 filter can look over a 2 sequential words in a piece of text, i.e. a bi-gram. In the previous tutorial we looked at the FastText model which used bi-grams by explicitly adding them to the end of a text, in this CNN model we will instead use multiple filters of different sizes which will look at the bi-grams (a 1x2 filter), tri-grams (a 1x3 filter) and n-grams (a 1x$n$ filter) within the text.

The intuition here is that the appearance of certain bi-grams, tri-grams and n-grams within the review will be a good indication of the final sentiment.

## Preparing Data

As in the previous notebooks, we'll prepare the data. 

Unlike the previous notebook with the FastText model, we no longer explicitly need to create the bi-grams and append them to the end of the sentence.

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('all/train.csv')
train, valid = train_test_split(df, test_size=0.2)

In [40]:
train.to_csv("all/train_split.csv", index=False)
valid.to_csv("all/valid_split.csv", index=False)

In [1]:
import re
import spacy
NLP = spacy.load('en')
MAX_CHARS = 20000
def tokenizer(comment):
    comment = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", 
        str(comment))
    comment = re.sub(r"[ ]+", " ", comment)
    comment = re.sub(r"\!+", "!", comment)
    comment = re.sub(r"\,+", ",", comment)
    comment = re.sub(r"\?+", "?", comment)
    if (len(comment) > MAX_CHARS):
        comment = comment[:MAX_CHARS]
    return [
        x.text for x in NLP.tokenizer(comment) if x.text != " "]

In [41]:
import torch
from torchtext import data
from torchtext import datasets
import random
import logging

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

LOGGER = logging.getLogger("sentiment_dataset")
LOGGER.debug("Preparing CSV files...")

TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True)
LABEL = data.LabelField(dtype=torch.float)

fields = [(None, None), ('text', TEXT), ('label', LABEL)]

LOGGER.debug("Reading train csv file...")


train_data = data.TabularDataset(
        path='all/train_split.csv', format='csv', 
        skip_header=True,
        fields=fields)

In [42]:
print(vars(train_data[0]))

{'text': ['great', 'hotel', 'great', 'location', '.', 'family', 'stayed', 'days', 'early', 'june', '.', 'quaint', 'hotel', 'spacious', 'quad', 'rooms', 'double', 'beds', 'desk', 'tv', 'mini', 'bar', 'big', 'bathroom', 'shower', 'toiletries', 'hairdryer', 'amenities', 'larger', 'hotel', '.', 'staff', 'helpful', 'friendly', 'time', 'email', 'check', 'out', '.', 'elevator', 'tiny', 'prepared', 'multiple', 'trips', 'luggage', 'stairs', '.', 'kids', 'enjoyed', 'running', 'flights', 'stairs', '.', 'location', 'great', 'just', 'minute', 'walk', 'eiffel', 'tower', 'champs', 'mer', '10', 'minutes', 'rue', 'cler', 'metro', 'stations', 'five', 'minute', 'walk', 'multiple', 'bakeries', 'cafes', '.', 'good', 'continental', 'breakfast', 'available', 'hotel.in', 'addition', 'price', 'reasonable', '150', 'euros', 'night', '.', 'definitely', 'stay', '.'], 'label': '5'}


In [43]:
valid_data = data.TabularDataset(
        path='all/valid_split.csv', format='csv', 
        skip_header=True,
        fields=fields)

In [44]:
print(vars(valid_data[0]))

{'text': ['reasonable', 'place', 'stay', 'stayed', '2004', 'elevator', 'not', 'working', '.', 'pleased', 'hotel', 'great', 'central', 'location', 'close', 'train', 'station', 'rental', 'car', 'got', 'later', 'trip', 'large', 'room', 'clean', 'everytbhing', 'worked', 'helpful', 'staff', 'not', 'expensive', 'given', 'market', 'florence', 'stay', '.'], 'label': '4'}


In [ ]:
test_data = data.TabularDataset(
        path='all/test.csv', format='csv', 
        skip_header=True,
        fields=[
            ('id', None),
            ('text', TEXT),
            ('label', None)
        ])

In [10]:
print(vars(test_data[0]))

{'text': ['great', 'hotel', 'business', 'travellers', 'couples', 'travel', 'beijing', 'stayed', 'hotels', 'years', 'great', '.', 'attentive', 'service', 'clean', 'comfortable', 'rooms', 'friendly', 'staff', 'good', 'set', 'restaurants', '.', 'lots', 'really', 'good', 'restaurants', 'close', 'local', 'flavor', 'especially', 'hot', 'pots', 'peking', 'duck', '.', 'concierge', 'knowledgeable', 'helpful', '.', 'terrific', 'beds', 'pillows', 'great', 'bath', 'robes', '.', 'good', 'value', 'money', '.']}


Build the vocab and load the pre-trained word embeddings.

In [11]:
TEXT.build_vocab(train_data, valid_data, test_data, max_size=16000, vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [05:20, 3.11MB/s]                               
100%|█████████▉| 398422/400000 [00:30<00:00, 13266.20it/s]

In [13]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 250552), ('hotel', 48994), ('room', 34653), ('not', 31617), ('great', 21170), ("n't", 18993), ('good', 17074), ('staff', 16251), ('stay', 15175), ('did', 14618), ('just', 12615), ('nice', 12450), ('rooms', 12079), ('no', 11744), ('location', 11091), ('stayed', 10466), ('service', 10078), ('night', 9893), ('time', 9893), ('beach', 9755)]


In [14]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x11000b488>, {'5': 0, '4': 1, '3': 2, '2': 3, '1': 4})


As before, we create the iterators.

In [25]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.Iterator.splits(
    (train_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch=False,
    repeat=False,
    device=device)

In [26]:
print(len(TEXT.vocab))

16002


## Build the Model

Now to build our model.

The first major hurdle is visualizing how CNNs are used for text. Images are typically 2 dimensional (we'll ignore the fact that there is a third "colour" dimension for now) whereas text is 1 dimensional. However, we know that the first step in almost all of our previous tutorials (and pretty much all NLP pipelines) is converting the words into word embeddings. This is how we can visualize our words in 2 dimensions, each word along one axis and the elements of vectors aross the other dimension. Consider the 2 dimensional representation of the embedded sentence below:

![](https://i.imgur.com/ci1h9hv.png)

We can then use a filter that is **[n x emb_dim]**. This will cover $n$ sequential words entirely, as their width will be `emb_dim` dimensions. Consider the image below, with our word vectors are represented in green. Here we have 4 words with 5 dimensional embeddings, creating a [4x5] "image" tensor. A filter that covers two words at a time (i.e. bi-grams) will be **[2x5]** filter, shown in yellow, and each element of the filter with have a _weight_ associated with it. The output of this filter (shown in red) will be a single real number that is the weighted sum of all elements covered by the filter.

![](https://i.imgur.com/QlXduXu.png)

The filter then moves "down" the image (or across the sentence) to cover the next bi-gram and another output (weighted sum) is calculated. 

![](https://i.imgur.com/wuA330x.png)

Finally, the filter moves down again and the final output for this filter is calculated.

![](https://i.imgur.com/gi1GaEz.png)

In our case (and in the general case where the width of the filter equals the width of the "image"), our output will be a vector with number of elements equal to the height of the image (or lenth of the word) minus the height of the filter plus one, $4-2+1=3$ in this case.

This example showed how to calculate the output of one filter. Our model (and pretty much all CNNs) will have lots of these filters. The idea is that each filter will learn a different feature to extract. In the scenario of analysing text, we are hoping each of the **[2 x emb_dim]** filters will be looking for the occurence of different bi-grams. 

In our model, we will also have different sizes of filters, heights of 3, 4 and 5, with 100 of each of them. The intuition is that we will be looking for the occurence of different tri-grams, 4-grams and 5-grams that are relevant for analysing sentiment of movie reviews.

The next step in our model is to use *pooling* (specifically *max pooling*) on the output of the convolutional layers. This is similar to the FastText model where we performed the average over each of the word vectors, implemented by the `F.avg_pool2d` function, however instead of taking the average over a dimension, we are taking the maximum value over a dimension. Below an example of taking the maximum value (0.9) from the output of the convolutional layer on the example sentence (not shown in the activation function applied to the output of the convolutions).

![](https://i.imgur.com/gzkS3ze.png)

The idea here is that the maximum value is the "most important" feature for determining the sentiment of the review, which corresponds to the "most important" n-gram within the review. How do we know what the "most important" n-gram is? Luckily, we don't have to! Through backpropagation, the weights of the filters are changed so that whenever certain n-grams that are highly indicative of the sentiment are seen, the output of the filter is a "high" value. This "high" value then passes through the max pooling layer if it is the maximum value in the output. 

As our model has 100 filters of 3 different sizes, that means we have 300 different n-grams the model thinks are important. We concatenate these together into a single vector and pass them through a linear layer to predict the sentiment. We can think of the weights of this linear layer as "weighting up the evidence" from each of the 300 n-grams and making a final decision. 

### Implementation Details

We implement the convolutional layers with `nn.Conv2d`. The `in_channels` argument is the number of "channels" in your image going into the convolutional layer. In actual images this is usually 3 (one channel for each of the red, blue and green channels), however when using text we only have a single channel, the text itself. The `out_channels` is the number of filters and the `kernel_size` is the size of the filters. Each of our `kernel_size`s is going to be **[n x emb_dim]** where $n$ is the size of the n-grams.

In PyTorch, RNNs want the input with the batch dimension second, whereas CNNs want the batch dimension first. Thus, the first thing we do to our input is `permute` it to make it the correct shape. We then pass the sentence through an embedding layer to get our embeddings. The second dimension of the input into a `nn.Conv2d` layer must be the channel dimension. As text technically does not have a channel dimension, we `unsqueeze` our tensor to create one. This matches with our `in_channels=1` in the initialization of our convolutional layers. 

We then pass the tensors through the convolutional and pooling layers, using the `ReLU` activation function after the convolutional layers. Another nice feature of the pooling layers is that they handle sentences of different lengths. The size of the output of the convolutional layer is dependent on the size of the input to it, and different batches contain sentences of different lengths. Without the max pooling layer the input to our linear layer would depend on the size of the input sentence (not what we want). One option to rectify this would be to trim/pad all sentences to the same length, however with the max pooling layer we always know the input to the linear layer will be the total number of filters. **Note**: there an exception to this if your sentence(s) are shorter than the largest filter used. You will then have to pad your sentences to the length of the largest filter. In the IMDb data there are no reviews only 5 words long so we don't have to worry about that, but you will if you are using your own data.

Finally, we perform dropout on the concatenated filter outputs and then pass them through a linear layer to make our predictions.

Currently the `CNN` model can only use 3 different sized filters, but we can actually improve the code of our model to make it more generic and take any number of filters.

We do this by placing all of our convolutional layers in a  `nn.ModuleList`, a function used to hold a list of PyTorch `nn.Module`s. If we simply used a standard Python list, the modules within the list cannot be "seen" by any modules outside the list which will cause us some errors.

We can now pass an arbitrary sized list of filter sizes and the list comprehension will create a convolutional layer for each of them. Then, in the `forward` method we iterate through the list applying each convolutional layer to get a list of convolutional outputs, which we also feed through the max pooling in a list comprehension before concatenating together and passing through the dropout and linear layers.

In [29]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(fs,embedding_dim)) for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #x = [sent len, batch size]
        
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

We create an instance of our `CNN` class.

In [30]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

And load the pre-trained embeddings

In [31]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 0.1810,  0.0101,  0.3064,  ..., -0.0590, -0.2692,  0.4337],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2541,  0.1689,  0.7700,  ...,  0.0157,  0.1503,  0.3728]])

## Train the Model

Training is the same as before. We initialize the optimizer, loss function (criterion) and place the model and criterion on the GPU (if available)

In [32]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

We implement the function to calculate accuracy...

In [35]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

We define a function for training our model...

**Note**: as we are using dropout again, we must remember to use `model.train()` to ensure the dropout is "turned on" while training.

In [36]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
                
        loss = criterion(predictions, batch.label)
                        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We define a function for testing our model...

**Note**: again, as we are now using dropout, we must remember to use `model.eval()` to ensure the dropout is "turned off" while evaluating.

In [37]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, we train our model...

In [191]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss = train(model, train_iterator, optimizer, criterion)
#     valid_loss = evaluate(model, valid_iterator, criterion)
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} |')
#     print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

0.7396591901779175
0.5295121669769287
-0.04272681847214699
0.02462313324213028
0.48097139596939087
0.0253620482981205
0.4826509654521942
0.07931263744831085
-0.4978101849555969
-1.5923289060592651
-0.909570038318634
-0.32911938428878784
0.6869925856590271
-0.836698055267334
-1.2578777074813843
-0.970185399055481
-1.2556567192077637
-1.0906723737716675
-0.8986856937408447
0.22032839059829712
1.0518012046813965
-1.1935312747955322
-0.7524811625480652
-0.09380757808685303
-1.1993309259414673
-4.334022521972656
0.11572204530239105
-1.292037010192871
-0.7329620718955994
-2.042145252227783
-0.9803744554519653
-1.804610252380371
0.2942756116390228
-3.7640774250030518
-3.9344589710235596
0.16779345273971558
-3.341644525527954
-3.590407133102417
-6.655394554138184
0.2617572546005249
-4.968047142028809
-7.161271572113037
2.2938077449798584
-3.0122199058532715
-0.7612799406051636
-3.0848073959350586
-2.1895363330841064
-1.7003438472747803
-2.248624324798584
-10.051241874694824
0.2648579180240631


-828.61328125
-1363.3099365234375
-1138.95556640625
-778.9169311523438
-2128.271728515625
-397.2525939941406
-985.2772216796875
-912.27392578125
-2988.855224609375
-628.27490234375
-590.6398315429688
-1476.0821533203125
-1142.614501953125
-1316.176513671875
-635.2007446289062
-1264.1053466796875
-920.39208984375
-2486.15380859375
-1854.6561279296875
-2290.81005859375
-1334.775146484375
-2394.666015625
-2336.623291015625
-2447.6630859375
-2442.607177734375
-541.7891235351562
-1130.9278564453125
-1355.2763671875
-830.0490112304688
-2095.1337890625
-765.7333374023438
-3397.49609375
-3386.0859375
-455.5518493652344
-1841.7705078125
-1158.3638916015625
-536.5801391601562
-1838.3155517578125
-1073.82080078125
-50.75775909423828
-2398.63623046875
-3107.857177734375
-2461.11279296875
-1244.796630859375
-2628.99951171875
-3838.151611328125
-2577.733642578125
-1896.6737060546875
-1701.447021484375
-1239.3939208984375
-2990.527587890625
-2218.776611328125
-772.2772216796875
-1095.2169189453125
-1

-24717.27734375
-24534.787109375
-15723.78515625
-11569.29296875
-8878.48046875
-6287.5224609375
-3038.386474609375
-18629.037109375
-2475.902099609375
-7932.935546875
-17586.76953125
-15132.818359375
-18742.814453125
-3097.3583984375
-26483.373046875
-7857.5009765625
-15322.0078125
-15765.169921875
-2516.374755859375
-14801.833984375
-13629.65625
-14622.60546875
-20472.9296875
-11783.861328125
-5994.20654296875
-25828.0
-21497.13671875
-4610.8203125
-16749.921875
-22449.115234375
-12280.9765625
-25659.90234375
-16135.671875
-6482.1640625
-14839.3408203125
-16732.21875
-4920.23486328125
-29672.765625
-12756.6728515625
-13901.583984375
-24597.7109375
-11865.546875
-22618.599609375
-19968.45703125
-34749.37109375
-15766.2216796875
-4891.1435546875
-3280.69970703125
-20009.890625
-20814.373046875
-24487.7734375
-27882.43359375
-22586.4375
-13231.669921875
-15685.7255859375
-20260.111328125
-22699.328125
-24218.033203125
-14135.65625
-11007.6181640625
-19908.06640625
-7015.8203125
-11411.6

...and get our best test accuracy yet! 

In [193]:
test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

TypeError: '<' not supported between instances of 'Example' and 'Example'

## User Input

And again, as a sanity check we can check some input sentences

**Note**: As mentioned in the implementation details, the input sentence has to be at least as long as the largest filter height used. We modify our `predict_sentiment` function to also accept a minimum length argument. If the tokenized input sentence is less than `min_len` tokens, we append padding tokens (`<pad>`) to make it `min_len` tokens.

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(sentence, min_len=5):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

An example negative review...

In [ ]:
predict_sentiment("This film is terrible")

An example positive review...

In [ ]:
predict_sentiment("This film is great")